In [1]:
#!/usr/bin/env Rscript

# Load libraries
require(ggplot2)
require(reshape2)

# Step 1: Read the CSV file
bids <- "/om2/user/mabdel03/files/Ravi_ISO_MRI/reformatted/"
read_csv <- file.path(bids, "code", "slm_data2.csv")
data <- read.csv(read_csv)
colnames <- names(data)

# Extract subsets with QC
fmri_data <- subset(data, fmri_qc == 1)
dwi_data <- subset(data, dwi_qc == 1)
anat_data <- subset(data, anat_qc == 1)

# Set working directory
setwd(file.path(bids, "code", "model_outputs"))


Loading required package: ggplot2

Loading required package: reshape2



In [43]:
run_ols_model <- function(dataframe, main_predictor, main_dependent, covariates_numeric = NULL, covariates_factor = NULL) {
  # Selecting relevant columns from the dataframe
  data_subset <- dataframe[, c(main_predictor, main_dependent, covariates_numeric, covariates_factor)]
  
  # Remove rows with missing values
  data_subset <- na.omit(data_subset)
  
  # Z-scoring main predictor and dependent variable along with numeric covariates if they are not empty
  numeric_cols <- c(main_predictor, main_dependent, covariates_numeric)
  if (!is.null(numeric_cols)) {
    data_subset[numeric_cols] <- scale(data_subset[numeric_cols])
  }
  
  # Converting factor variables to factor type
  if (!is.null(covariates_factor)) {
    for (factor_var in covariates_factor) {
      data_subset[[factor_var]] <- factor(data_subset[[factor_var]])
    }
  }
  
  # Constructing formula strings
  if (is.null(covariates_numeric) && is.null(covariates_factor)) {
    formula_string_full <- paste(main_dependent, "~", main_predictor)
    formula_string_reduced <- formula_string_full
  } else {
    formula_terms <- c(main_predictor, covariates_numeric, covariates_factor)
    formula_string_full <- paste(main_dependent, "~", paste(formula_terms, collapse = " + "))
    formula_string_reduced <- paste(main_dependent, "~", paste(setdiff(formula_terms, main_predictor), collapse = " + "))
  }
  
  # Running the full OLS model
  ols_model_full <- lm(formula_string_full, data = data_subset)
  
  # Running the reduced OLS model
  ols_model_reduced <- lm(formula_string_reduced, data = data_subset)
  
  # Calculating residuals from the full model
  #residuals_full <- residuals(ols_model_full)
  
  # Plotting main predictor vs residuals
  #plot(data_subset[[main_predictor]], residuals_full, xlab = main_predictor, ylab = "Residuals (Actual - Predicted)", main = "Main Predictor vs Residuals")
  
  # Adding best fit line
  #abline(lm(residuals_full ~ data_subset[[main_predictor]]), col = "red")
  
  # Adding confidence intervals around the best fit line
  #lines(predict(lm(residuals_full ~ data_subset[[main_predictor]]), interval = "confidence"), col = "blue", lty = 2)
  
  # Calculating adjusted R-squared for the full model
  adj_rsq_full <- summary(ols_model_full)$adj.r.squared
  
  # Calculating adjusted R-squared for the reduced model
  adj_rsq_reduced <- summary(ols_model_reduced)$adj.r.squared
  
  # Calculating change in adjusted R-squared
  delta_adj_rsq <- adj_rsq_full - adj_rsq_reduced
  
  # Printing formula for the full model
  cat("Formula (Full Model):", formula_string_full, "\n\n")
  cat("Formula (Reduced Model):", formula_string_reduced, "\n\n")
  
  # Printing model summary for the full model
  cat("Model Summary (Full Model):\n")
  print(summary(ols_model_full))
  
  # Printing change in adjusted R-squared
  cat("\nChange in Adjusted R-squared (Full vs Reduced Model):", delta_adj_rsq, "\n\n")
  
  # Returning the OLS model object for the full model
  return(list(model = ols_model_full, delta_adj_rsq = delta_adj_rsq))
}


In [41]:
string_to_match <- "dwi.*white.*"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]
columns_match

[1] "dwi_white_matter_average_ad"     "dwi_white_matter_average_dti_fa"
[3] "dwi_white_matter_average_md"     "dwi_white_matter_average_qa"    
[5] "dwi_white_matter_average_rd"

In [44]:
# Run a model
data_model <- dwi_data
string_to_match <- "dwi.*AAL.*Frontal.*_dti_fa"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]
for (main_dependent in columns_match) {
    main_predictor <- "pheno_social_isolation"
    covariates_numeric <- c("pheno_age_at_visit", "dwi_white_matter_average_dti_fa")
    covariates_factor <- c("pheno_msex")
    model <- run_ols_model(dataframe=data_model,
                  main_dependent = main_dependent,
                  main_predictor = main_predictor,
                  covariates_numeric = covariates_numeric,
                  covariates_factor = covariates_factor)
    }
string_to_match <- "regionsurfacestats.*AAL.*Frontal.*Vol"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]

Formula (Full Model): dwi_AAL_Frontal_Sup_L_dti_fa ~ pheno_social_isolation + pheno_age_at_visit + dwi_white_matter_average_dti_fa + pheno_msex 

Formula (Reduced Model): dwi_AAL_Frontal_Sup_L_dti_fa ~ pheno_age_at_visit + dwi_white_matter_average_dti_fa + pheno_msex 

Model Summary (Full Model):

Call:
lm(formula = formula_string_full, data = data_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.2942 -0.5122  0.0060  0.4809  4.4834 

Coefficients:
                                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)                      0.030996   0.037232   0.833   0.4055    
pheno_social_isolation           0.004964   0.033209   0.149   0.8812    
pheno_age_at_visit              -0.056332   0.034328  -1.641   0.1014    
dwi_white_matter_average_dti_fa  0.626195   0.034150  18.337   <2e-16 ***
pheno_msex1                     -0.130044   0.076449  -1.701   0.0895 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard er

In [35]:
# Run a model
data_model <- anat_data
#main_dependent <- "pheno_social_isolation"
#main_predictor <- "bold_AAL_Precuneus_L_alff"
#covariates_numeric <- c("pheno_age_at_visit",'bold_wholebrain_average_alff')
#covariates_factor <- c("pheno_msex")
# Find all columns with "bold_AAL_XXXXX_t1t2ratio"
string_to_match <- "regionsurfacestats.*AAL.*Frontal.*SurfArea*"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]
for (main_dependent in columns_match) {
    main_predictor <- "pheno_social_isolation"
    covariates_numeric <- c("pheno_age_at_visit","brainmeasures_EstimatedTotalIntraCranialVol_eTIV")
    covariates_factor <- c("pheno_msex")
    model <- run_ols_model(dataframe=data_model,
                  main_dependent = main_dependent,
                  main_predictor = main_predictor,
                  covariates_numeric = covariates_numeric,
                  covariates_factor = covariates_factor)
    }
string_to_match <- "regionsurfacestats.*AAL.*Frontal.*Vol"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]

Formula (Full Model): regionsurfacestats_AAL_lh_Frontal_Sup_L_SurfArea ~ pheno_social_isolation + pheno_age_at_visit + brainmeasures_EstimatedTotalIntraCranialVol_eTIV + pheno_msex 

Model Summary (Full Model):

Call:
lm(formula = formula_string_full, data = data_subset)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.02724 -0.49164 -0.04401  0.48192  2.31438 

Coefficients:
                                                 Estimate Std. Error t value
(Intercept)                                      -0.03504    0.03842  -0.912
pheno_social_isolation                           -0.02878    0.03333  -0.864
pheno_age_at_visit                               -0.32083    0.03367  -9.528
brainmeasures_EstimatedTotalIntraCranialVol_eTIV  0.60332    0.03929  15.357
pheno_msex1                                       0.16651    0.09552   1.743
                                                 Pr(>|t|)    
(Intercept)                                        0.3623    
pheno_social_isolation 

In [20]:
# Run a model
data_model <- anat_data
#main_dependent <- "pheno_social_isolation"
#main_predictor <- "bold_AAL_Precuneus_L_alff"
#covariates_numeric <- c("pheno_age_at_visit",'bold_wholebrain_average_alff')
#covariates_factor <- c("pheno_msex")
# Find all columns with "bold_AAL_XXXXX_t1t2ratio"
string_to_match <- "bold.*Frontal.*t1t2ratio"
columns_match <- names(data_model)[grep(string_to_match, colnames(data_model))]
for (main_dependent in columns_match) {
    main_predictor <- "pheno_social_isolation"
    covariates_numeric <- c("pheno_age_at_visit")
    covariates_factor <- c("pheno_msex")
    model <- run_ols_model(dataframe=data_model,
                  main_dependent = main_dependent,
                  main_predictor = main_predictor,
                  covariates_numeric = covariates_numeric,
                  covariates_factor = covariates_factor)
    }

Formula (Full Model): bold_AAL_Frontal_Sup_L_t1t2ratio ~ pheno_social_isolation + pheno_age_at_visit + pheno_msex 

Model Summary (Full Model):

Call:
lm(formula = formula_string_full, data = data_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.1691 -0.0844 -0.0513 -0.0194 22.0636 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)
(Intercept)             0.012303   0.050889   0.242    0.809
pheno_social_isolation -0.009493   0.046080  -0.206    0.837
pheno_age_at_visit      0.041877   0.046017   0.910    0.363
pheno_msex1            -0.057767   0.110475  -0.523    0.601

Residual standard error: 1.002 on 489 degrees of freedom
Multiple R-squared:  0.002332,	Adjusted R-squared:  -0.003789 
F-statistic: 0.3809 on 3 and 489 DF,  p-value: 0.7668


Change in Adjusted R-squared (Full vs Reduced Model): -0.001961612 

Formula (Full Model): bold_AAL_Frontal_Sup_R_t1t2ratio ~ pheno_social_isolation + pheno_age_at_visit + pheno_msex 

Model Summary (Full